- Augmentations are done on-the-fly with **set_transform**
- Set __getitem__ return format using this transform. The transform is applied on-the-fly on batches when __getitem__ is called. As set_format(), this can be reset using reset_format().
- For spectrogram augmentations we use torchaudio (https://pytorch.org/audio/stable/transforms.html)
- For waveform augmentations we use torch-audiomentations (https://github.com/asteroid-team/torch-audiomentations)

# Imports

In [ ]:
from datasets import load_dataset

import librosa
import matplotlib.pyplot as plt

import torch
from torchaudio import transforms

import IPython.display as ipd

#### Augmentation module for our datapipeline

In [2]:
from birdset.augmentations import AudioAugmentor

## Create a Huggingface dataset

#### For simplicity, we use the ESC50 dataset

In [3]:
esc = load_dataset("ashraq/esc50", split="train")

Found cached dataset parquet (/mnt/home/rheinrich/.cache/huggingface/datasets/ashraq___parquet/ashraq--esc50-1000c3b73cc1500f/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


#### Define a function for normalizing the audio signals

Here, however, we would first have to calculate the mean and standard deviation for the entire training data set!

In [4]:
def normalize_audio(audio_tensor, mean, std):
    return (audio_tensor-mean) / std

#### Define a general preprocessing function

In this function we can combine data augmentations, normalization, etc.

In [ ]:
def preprocess(waveform, 
               use_spectrogram,
               spectrogram_augmentations=None,
               waveform_augmentations=None,
               n_fft=1024,
               hop_length=512,
               n_mels = None,
              ):
    audio_augmentor = AudioAugmentor(sample_rate = waveform["sampling_rate"], 
                                     use_spectrogram=use_spectrogram,
                                     spectrogram_augmentations = spectrogram_augmentations,
                                     waveform_augmentations = waveform_augmentations,
                                     n_fft=n_fft,
                                     hop_length=hop_length,
                                     n_mels=n_mels,
                                     db_scale=True)
    audio_augmented = audio_augmentor.combined_augmentations(waveform["array"])
    
    #audio_augmented = normalize_audio(audio_augmented)
    return audio_augmented

# Waveform data augmentations

#### Create a dictionary which defines the augmentation techniques to be used as well as their respective parameters

In [6]:
# path to the aufio files which are used for creating background noise
speech_command_path = "/mnt/home/rheinrich/deep_bird_detect/datapipeline/speech_command_dataset"

# dictionary which defines the waveform data augmentations to be used
waveform_augmentations_dict = {"colored_noise": {"prob": 0.5, "min_snr_in_db": 3.0, "max_snr_in_db": 30.0,
                                                 "min_f_decay": -2.0, "max_f_decay": 2.0,},
                               "background_noise": {"background_paths": speech_command_path ,"prob":0.5,
                                                    "min_snr_in_db": 3.0, "max_snr_in_db": 30.0,},
                               "pitch_shift": {"prob": 0.5, "min_transpose_semitones": -4.0, "max_transpose_semitones": 4.0,},
                               "time_mask": {"prob": 0.5, "min_band_part": 0.0, "max_band_part": 0.5},
                               "time_stretch": {"prob": 0.5, "min_rate": 0.8, "max_rate": 1.25},
                              }

#### Create a function that applies our prepocessing and can be passed to the set_transform function of the dataset

In [7]:
def train_transforms_waveform(examples):
    examples["audio_array_preprocessed"] = [preprocess(waveform=audio, waveform_augmentations=waveform_augmentations_dict, use_spectrogram=False) for audio in examples["audio"]]
    return examples

In [8]:
esc.set_transform(train_transforms_waveform)

### Results

#### waveform audio data without augmentations

In [9]:
waveform = esc[101]["audio"]["array"]
sr = esc[101]["audio"]["sampling_rate"]
ipd.Audio(data=waveform, rate=sr)

#### waveform audio data with augmentations - nice, the augmentations are audible!

In [10]:
waveform_augmented = esc[101]["audio_array_preprocessed"]
sr = esc[101]["audio"]["sampling_rate"]
ipd.Audio(data=waveform_augmented, rate=sr)

# Spectrogram and waveform data augmentations combined

#### Create a HuggingFace dataset (ESC50)

In [11]:
esc = load_dataset("ashraq/esc50", split="train")

Found cached dataset parquet (/mnt/home/rheinrich/.cache/huggingface/datasets/ashraq___parquet/ashraq--esc50-1000c3b73cc1500f/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


#### Create dictionaries which define the data augmentation techniques to be used as well as their respective parameters

In [12]:
# path to the aufio files which are used for creating background noise
speech_command_path = "/mnt/home/rheinrich/deep_bird_detect/datapipeline/speech_command_dataset"

# dictionary which defines the waveform data augmentations to be used
waveform_augmentations_dict = {"colored_noise": {"prob": 0.5, "min_snr_in_db": 3.0, "max_snr_in_db": 30.0,
                                                 "min_f_decay": -2.0, "max_f_decay": 2.0,},
                               "background_noise": {"background_paths": speech_command_path ,"prob":0.5,
                                                    "min_snr_in_db": 3.0, "max_snr_in_db": 30.0,},
                               "pitch_shift": {"prob": 0.5, "min_transpose_semitones": -4.0, "max_transpose_semitones": 4.0,},
                              }

In [13]:
# dictionary which defines the spectrogram data augmentations to be used
spectrogram_augmentations_dict = {"time_masking": {"time_mask_param": 100, "prob": 0.5},
                                  "frequency_masking": {"freq_mask_param": 100, "prob": 0.5},
                                  "time_stretch": {"prob": 0.33, "min_rate": 0.8, "max_rate": 1.25,}
                                 }

#### Create a function that applies our prepocessing and can be passed to the set_transform function of the dataset

In [14]:
def train_transforms_with_spec(examples):
    examples["audio_array_preprocessed"] = [preprocess(waveform=audio, use_spectrogram=True, spectrogram_augmentations=spectrogram_augmentations_dict, waveform_augmentations=waveform_augmentations_dict) for audio in examples["audio"]]
    return examples

In [15]:
esc.set_transform(train_transforms_with_spec)

### Results

In [16]:
spectrogram=esc[101]["audio_array_preprocessed"]

#### spectrogram with augmentations

In [17]:
plt.imshow(spectrogram.squeeze().numpy())

#### Convert the spectrogram back to waveform audio data

The GriffinLim transformation converts power spectrograms back to waveform.

If Mel-scale spectrograms are used instead, the InverseMelScale transformation has to be applied first, see https://pytorch.org/audio/stable/transforms.html.

In [18]:
transform_to_waveform = transforms.GriffinLim(n_fft=1024, hop_length=512)

Note: since the spectrogram was converted to decibel (dB) units in the AudioAugmentor, we also need to invert the dB-scale transformation.

In [19]:
spectrogram_unscaled = spectrogram.numpy()
spectrogram_unscaled = librosa.db_to_power(spectrogram_unscaled)
spectrogram_unscaled = torch.from_numpy(spectrogram_unscaled)
waveform_from_spectrogram = transform_to_waveform(spectrogram_unscaled)

#### the waveform obtained from the augmented spectrogram.

In [20]:
sr = esc[101]["audio"]["sampling_rate"]
ipd.Audio(data=waveform_from_spectrogram, rate=sr)

# Create a PyTorch dataloader from the HuggingFace dataset

In [21]:
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

In [22]:
def collate_batch(batch):
    input_features = [x["audio_array_preprocessed"] for x in batch]
    input_features = torch.cat(input_features, 0)
    targets = [x["target"] for x in batch]
    targets = torch.Tensor(targets)
    return input_features, targets

In [23]:
dataloader = DataLoader(esc, batch_size=5, shuffle=True, collate_fn = collate_batch)

In [24]:
for i in dataloader: 
    print(i)
    break

(tensor([[[ 0.9085,  1.4793,  1.0389,  ...,  1.2960,  1.7056,  1.7235],
         [ 1.2333,  1.5261,  1.7780,  ...,  1.9140,  0.4410,  1.0365],
         [ 1.2614,  1.0216,  1.2542,  ...,  1.5317,  1.2378,  0.1484],
         ...,
         [-4.3052, -3.2671, -3.6737,  ..., -3.7132, -5.3129, -3.7536],
         [-3.8896, -3.8300, -5.5740,  ..., -3.1346, -4.0516, -4.0376],
         [-3.1665, -4.9380, -5.1289,  ..., -3.1226, -4.9468, -4.6243]],

        [[ 3.3213, -1.1602,  1.5763,  ...,  1.8396,  2.3336,  2.2096],
         [ 2.9855,  2.4760,  3.0326,  ...,  2.8038,  2.4690,  2.3985],
         [ 0.8365,  2.8021,  3.1691,  ...,  3.6942,  2.1839,  2.1576],
         ...,
         [-1.0990, -2.6500, -1.9091,  ..., -2.5233, -1.4699, -2.1597],
         [-1.4715, -1.9487, -2.6993,  ..., -2.4759, -1.6192, -1.7414],
         [-2.3496, -1.3249, -1.8528,  ..., -2.7282, -3.5941, -3.4955]],

        [[ 0.9818,  0.7401, -0.0721,  ...,  1.3722, -1.3037,  1.6600],
         [-0.2923, -0.3187,  0.1465,  ...,  